In [1]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
from global_land_mask import globe

In [2]:
#####Constants
Cp = 1004           #J/kg/K
Rd = 287            #J/kg/K

In [3]:
import glob
merlist = np.sort(glob.glob('../data_merra/lat_30_70/new/MERRA2_*.nc'))
maclist = np.sort(glob.glob('../MACLWP_dailymean/take/wind1deg*.nc4'))

In [65]:
for i in range(len(merlist)): #len(maclist)
    d_path = merlist[i]
    data   = nc.Dataset(d_path)
    merlat = data.variables['lat'][:]
    merlon = data.variables['lon'][:]
    merlev = data.variables['lev'][:]
    merT   = data.variables['T'][:] #(time, lev, lat, lon)

for i in range(len(maclist)): 
    ddpath = maclist[i]
    ddata  = nc.Dataset(ddpath)
    macw   = ddata.variables['sfcwind'][:]
    maclat = ddata.variables['lat'][:]
    maclon = ddata.variables['lon'][:]
    

    

In [66]:
mac_lon = maclon-180.
mac_lon[mac_lon > 180] = mac_lon[mac_lon > 180]-360
mac_lon = np.array(mac_lon)
mer_lon = np.array(merlon)

In [67]:
mac_lat = np.array(maclat)
mer_lat = np.flip(merlat)
mer_lat = np.array(mer_lat)

In [70]:
mer_T = np.array(merT[:,:,::-1,:])